<a href="https://colab.research.google.com/github/worldofaryavart/colab_notebooks/blob/colabnotebook/making_scraperModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install playwright beautifulsoup4 pytesseract pillow PyMuPDF youtube_dl transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.8 MB/s eta 0:00:00


In [2]:
!playwright install chromium

164.5 MiB [] 0% 0.0s164.5 MiB [] 0% 5.3s164.5 MiB [] 0% 3.8s164.5 MiB [] 1% 3.2s164.5 MiB [] 2% 2.9s164.5 MiB [] 2% 3.0s164.5 MiB [] 3% 3.1s164.5 MiB [] 3% 3.2s164.5 MiB [] 4% 3.1s164.5 MiB [] 5% 3.0s164.5 MiB [] 5% 3.1s164.5 MiB [] 6% 2.7s164.5 MiB [] 7% 2.6s164.5 MiB [] 8% 2.5s164.5 MiB [] 9% 2.4s164.5 MiB [] 10% 2.4s164.5 MiB [] 11% 2.5s164.5 MiB [] 11% 2.6s164.5 MiB [] 12% 2.6s164.5 MiB [] 13% 2.6s164.5 MiB [] 14% 2.6s164.5 MiB [] 15% 2.6s164.5 MiB [] 16% 2.6s164.5 MiB [] 17% 2.5s164.5 MiB [] 18% 2.5s164.5 MiB [] 18% 2.4s164.5 MiB [] 19% 2.4s164.5 MiB [] 20% 2.4s164.5 MiB [] 21% 2.3s164.5 MiB [] 22% 2.3s164.5 MiB [] 23% 2.2s164.5 MiB [] 24% 2.2s164.5 MiB [] 25% 2.2s164.5 MiB [] 26% 2.2s164.5 MiB [] 26% 2.1s164.5 MiB [] 27% 2.1s164.5 MiB [] 28% 2.0s164.5 MiB [] 30% 1.9s164.5 MiB [] 31% 1.9s164.5 MiB [] 32% 1.8s164.5 MiB [] 33% 1.7s164.5 MiB [] 34% 1.7s164.5 MiB [] 35% 1.6s164.5 MiB [] 36% 1.6s164.5 MiB [] 38% 1.5s164.5 MiB [] 39% 1.5s164.5 MiB [] 40% 1.4s164.5 MiB [] 41% 1.4s164.5 M

In [3]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 2s (117 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [4]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,878 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123653 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [5]:
import asyncio
from asyncio import Queue
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from transformers import pipeline
from urllib.parse import urljoin, urlparse
import re
import time
import threading
import io
import aiohttp
import pytesseract
from PIL import Image
import fitz  # PyMuPDF
import warnings

In [6]:
!pip install nest_asyncio

In [7]:
import nest_asyncio

In [8]:
warnings.filterwarnings('ignore', category=FutureWarning)

nest_asyncio.apply()

In [9]:
class SuperPoweredCrawler:
    def __init__(self):
        self.visited_urls = set()
        self.url_queue = asyncio.Queue()  # Changed to asyncio.Queue
        self.results = []
        self.content_lock = threading.Lock()
        self.browser = None
        self.context = None  # Added context initialization

        # Initialize content extractors
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cpu")
        self.min_text_for_summary = 200

        # Configurable crawler settings
        self.max_depth = 3
        self.max_pages = 50
        self.max_pages_per_domain = 10
        self.concurrent_requests = 3

        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
        }

    async def initialize_browser(self):
        """Initialize browser and context"""
        playwright = await async_playwright().start()
        self.browser = await playwright.chromium.launch(headless=True)  # Fixed typo in chromium
        self.context = await self.browser.new_context(
            viewport={'width': 1200, 'height': 800},
            user_agent=self.headers['User-Agent'],
            extra_http_headers=self.headers  # Fixed parameter name
        )
        return playwright

    async def cleanup(self, playwright):
        """Cleanup browser resources"""
        if self.context:
            await self.context.close()
        if self.browser:
            await self.browser.close()
        await playwright.stop()

    async def start_crawl(self, seed_urls, search_query):
        """Start the crawling process with multiple seed URLs"""
        print(f"Starting crawl for query: {search_query}")

        # Initialize the queue with seed URLs
        for url in seed_urls:
            await self.url_queue.put((url, 0))

        try:
            playwright = await self.initialize_browser()
            workers = [self.crawler_worker(search_query) for _ in range(self.concurrent_requests)]
            await asyncio.gather(*workers)

        except Exception as e:
            print(f"Error during crawling: {str(e)}")

        finally:
            await self.cleanup(playwright)

        return self.results

    async def crawler_worker(self, search_query):
        """Worker process for crawling pages"""
        while True:
            try:
                if len(self.visited_urls) >= self.max_pages:
                    break

                try:
                    url, depth = await asyncio.wait_for(self.url_queue.get(), timeout=10)
                except asyncio.TimeoutError:
                    break

                if depth > self.max_depth or url in self.visited_urls:
                    self.url_queue.task_done()
                    continue

                print(f"Crawling: {url}")

                try:
                    page = await self.context.new_page()
                    response = await page.goto(
                        url,
                        wait_until='domcontentloaded',  # Fixed typo
                        timeout=30000
                    )

                    if not response:
                        print(f"No response from {url}")
                        await page.close()
                        self.url_queue.task_done()
                        continue

                    if response.status >= 400:
                        print(f"Error response from {url}: {response.status}")
                        await page.close()
                        self.url_queue.task_done()
                        continue

                    content = await page.content()
                    result = await self.process_page(page, response, content, url)  # Fixed parameter order
                    if result:
                        self.add_result(result, search_query)

                    if len(self.visited_urls) < self.max_pages:
                        new_urls = await self.extract_urls(page)
                        for new_url in new_urls:
                            if self.should_crawl(new_url):
                                await self.url_queue.put((new_url, depth + 1))

                    self.visited_urls.add(url)
                    await page.close()
                    await asyncio.sleep(2)

                except Exception as e:
                    print(f"Error processing {url}: {str(e)}")
                    if 'page' in locals():
                        await page.close()

                self.url_queue.task_done()

            except Exception as e:
                print(f"Worker error: {str(e)}")
                continue


    async def process_page(self, page, response, content, url):
        """Process page content based on content type"""
        try:
            content_type = response.headers.get('content-type', '').lower()

            if 'pdf' in content_type:
                return await self.process_pdf_content(response, url)
            elif any(img_type in content_type for img_type in ['image/jpeg', 'image/png', 'image/gif']):
                return await self.process_image_content(response, url)
            else:
                return await self.process_html_content(page, content, url)

        except Exception as e:
            print(f"Error processing page {url}: {str(e)}")
            return None

    def should_crawl(self, url):
      """Determine if a URL should be crawled"""
      try:
        parsed = urlparse(url)

        if not parsed.scheme in ['http', 'https']:
          return False

        domain = parsed.netloc
        domain_count = sum(1 for visited in self.visited_urls
                           if urlparse(visited).netloc == domain)
        if domain_count >= self.max_pages_per_domain:
          return False

        exclude_patterns = [
                r'\.(css|js|json|xml)$',
                r'(login|signup|logout)',
                r'(facebook|twitter|instagram)',
                r'\.(jpg|jpeg|png|gif)$',  # Skip direct image links
                r'\/api\/',
                r'\/rss\/',
                r'\/feed\/',
                r'\/search\?',
                r'\/page\/\d+',
            ]

        return not any(re.search(pattern, url, re.I)
                         for pattern in exclude_patterns)

      except:
            return False

    async def process_html_content(self, page, content, url):
        """
        Process HTML content
        """
        try:
            soup = BeautifulSoup(content, 'html.parser')

            # Extract text content
            text_content = ' '.join([p.get_text() for p in soup.find_all(['p', 'article', 'section'])])
            text_content = re.sub(r'\s+', ' ', text_content).strip()

            # Get title
            title = await page.title()

            # Generate summary if content is long enough
            summary = None
            if len(text_content) > self.min_text_for_summary:
                try:
                    # Calculate dynamic max_length based on content length
                    content_length = len(text_content.split())
                    max_length = min(150, content_length - 50)  # At least 50 tokens shorter than content
                    min_length = min(50, max_length - 20)  # At least 20 tokens shorter than max_length

                    if max_length > min_length:
                        summary = self.summarizer(
                            text_content[:4096],
                            max_length=max_length,
                            min_length=min_length
                        )[0]['summary_text']
                except Exception as e:
                    print(f"Error generating summary: {str(e)}")

            return {
                'url': url,
                'type': 'html',
                'title': title,
                'content': text_content[:5000],  # Limit content length
                'summary': summary,
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing HTML content for {url}: {str(e)}")
            return None

    async def process_pdf_content(self, response, url):
        """
        Process PDF content
        """
        try:
            pdf_data = await response.body()
            pdf_document = fitz.open(stream=pdf_data, filetype="pdf")
            text_content = ""

            for page_num in range(min(pdf_document.page_count, 10)):  # Limit to first 10 pages
                page = pdf_document[page_num]
                text_content += page.get_text()

            return {
                'url': url,
                'type': 'pdf',
                'content': text_content[:5000],  # Limit content length
                'page_count': pdf_document.page_count,
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing PDF {url}: {str(e)}")
            return None

    async def process_image_content(self, response, url):
        """
        Process image content
        """
        try:
            image_data = await response.body()
            image = Image.open(io.BytesIO(image_data))

            # Extract text using OCR
            try:
                ocr_text = pytesseract.image_to_string(image)
            except:
                ocr_text = ""

            return {
                'url': url,
                'type': 'image',
                'ocr_text': ocr_text,
                'metadata': {
                    'width': image.size[0],
                    'height': image.size[1],
                    'format': image.format
                },
                'timestamp': time.time()
            }

        except Exception as e:
            print(f"Error processing image {url}: {str(e)}")
            return None

    async def extract_urls(self, page):
        """
        Extract URLs from the page
        """
        try:
            # Get all links using JavaScript evaluation
            links = await page.evaluate('''() => {
                const links = Array.from(document.getElementsByTagName('a'));
                return links.map(link => link.href).filter(href => href);
            }''')

            return list(set(links))  # Remove duplicates

        except Exception as e:
            print(f"Error extracting URLs: {str(e)}")
            return []

    def add_result(self, result, search_query):
        """
        Add processed result to the results list with relevance scoring
        """
        with self.content_lock:
            result['relevance_score'] = self.calculate_relevance(result, search_query)
            self.results.append(result)
            self.results.sort(key=lambda x: x['relevance_score'], reverse=True)

    def calculate_relevance(self, result, query):
        """
        Calculate relevance score for a result
        """
        score = 0
        query_terms = query.lower().split()

        # Get the content to score
        content = ''
        if 'content' in result:
            content = result['content'].lower()
        elif 'ocr_text' in result:
            content = result['ocr_text'].lower()

        # Term frequency scoring
        for term in query_terms:
            score += content.count(term)

        # Type-based boosting
        type_boost = {
            'pdf': 1.2,
            'html': 1.0,
            'image': 0.8
        }
        score *= type_boost.get(result['type'], 1.0)

        return score


In [11]:
crawler = SuperPoweredCrawler()

seed_urls = [
    "https://research.ibm.com/quantum-computing",
    "https://www.nature.com/subjects/quantum-physics",
    "https://www.scientificamerican.com/computing/",
    "https://www.quantum-computing.news/",
    "https://quantumcomputing.stackexchange.com/"
]

results = await crawler.start_crawl(seed_urls, "quantum computing latest developments")

print("\nSearch Results:")
for i, result in enumerate(results[:10], 1):
    print(result)
    print(f"\n{i}. {result['url']}")
    print(f"Type: {result['type']}")
    print(f"Score: {result['relevance_score']:.2f}")
    if 'summary' in result and result['summary']:
        print(f"Summary: {result['summary']}")

Starting crawl for query: quantum computing latest developments
Crawling: https://research.ibm.com/quantum-computing
Crawling: https://www.nature.com/subjects/quantum-physics
Crawling: https://www.scientificamerican.com/computing/
Crawling: https://www.quantum-computing.news/
Error processing https://www.quantum-computing.news/: Page.goto: net::ERR_NAME_NOT_RESOLVED at https://www.quantum-computing.news/
Call log:
navigating to "https://www.quantum-computing.news/", waiting until "domcontentloaded"

Crawling: https://quantumcomputing.stackexchange.com/
Crawling: https://www.scientificamerican.com/article/has-generative-ai-lost-its-strange-charm/
Crawling: https://www.scientificamerican.com/computing/?page=2
Crawling: https://www.scientificamerican.com/getsciam/
Crawling: https://www.scientificamerican.com/article/a-computer-built-from-dna-can-find-patterns-in-photographs/
Crawling: https://www.scientificamerican.com/page/privacy-policy/
Crawling: https://www.scientificamerican.com/arti

In [12]:
print("\nSearch Results:")
for i, result in enumerate(results, 1):
    # print(result)
    print(f"\n{i}. {result['url']}")
    print(f"Type: {result['type']}")
    print(f"Score: {result['relevance_score']:.2f}")
    if 'summary' in result and result['summary']:
        print(f"Summary: {result['summary']}")


Search Results:

1. https://research.ibm.com/quantum-computing
Type: html
Score: 80.00
Summary: IBM offers cloud access to the most advanced quantum computers available. Learn, develop, and run programs with our quantum applications and systems. See more of our work on Quantum: Qiskit: Open-Source Quantum Development. Back to the page you came from.

2. https://www.nature.com/subjects/quantum-information
Type: html
Score: 32.00
Summary: Quantum information is problem solving and data processing using a quantum system as the information carrier. A method is introduced to compute provable bounds on noise-free quantum expectation values from noisy samples. Quantum information systems could be able to transmit data that is fundamentally secure and solve problems beyond the power of modern computers.

3. https://www.nature.com/subjects/quantum-physics
Type: html
Score: 28.00
Summary: Quantum physics is the study of matter and energy at its most fundamental level. A central tenet of quantum

In [10]:
async def run_crawler():
    """Main function to run the crawler"""
    crawler = SuperPoweredCrawler()

    seed_urls = [
        "https://research.ibm.com/quantum-computing",
        "https://www.nature.com/subjects/quantum-physics",
        "https://www.scientificamerican.com/computing/",
        "https://www.quantum-computing.news/",
        "https://quantumcomputing.stackexchange.com/"
    ]

    results = await crawler.start_crawl(seed_urls, "quantum computing latest developments")

    print("\nSearch Results:")
    for i, result in enumerate(results[:10], 1):
        print(f"\n{i}. {result['url']}")
        print(f"Type: {result['type']}")
        print(f"Score: {result['relevance_score']:.2f}")
        if 'summary' in result and result['summary']:
            print(f"Summary: {result['summary']}")

if __name__ == "__main__":
    asyncio.run(run_crawler())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Starting crawl for query: quantum computing latest developments
Crawling: https://research.ibm.com/quantum-computing
Crawling: https://www.nature.com/subjects/quantum-physics
Crawling: https://www.scientificamerican.com/computing/
Crawling: https://www.quantum-computing.news/
Crawling: https://quantumcomputing.stackexchange.com/
Crawling: https://www.nature.com/info/privacy
Error processing https://www.quantum-computing.news/: Page.goto: net::ERR_NAME_NOT_RESOLVED at https://www.quantum-computing.news/
Call log:
navigating to "https://www.quantum-computing.news/", waiting until "domcontentloaded"

Crawling: https://www.nature.com/natitaly
Crawling: https://www.nature.com/natafrica
Crawling: https://www.nature.com/nature
Crawling: https://www.nature.com/subjects/quantum-information
Crawling: https://partnerships.nature.com/product/branded-content-native-advertising/
Crawling: https://www.nature.com/articles/d41586-024-03376-4
Crawling: https://www.nature.com/info/terms-and-conditions
Cr